In [1]:
# Problems with graphs! -> moving down!
import dash
import pandas as pd
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
dataset = pd.read_csv('assets/web_data.csv')
app = dash.Dash(__name__, external_stylesheets=['assets/style.css'])
app.title= "Website analytics"
dataset.head()

,date,time,weekday,os,country,page
0,2021-10-01,00:00:00,Friday,Windows,United States,Detail
1,2021-10-01,00:00:00,Friday,Mac,United Kingdom,Shopping Cart
2,2021-10-01,00:00:00,Friday,Linux,Brazil,Detail
3,2021-10-01,00:00:00,Friday,Windows,Canada,Detail
4,2021-10-01,00:00:00,Friday,Linux,Germany,Landing


In [3]:
colors = ["#f2fffb", "#98ffe0", "#6df0c8", "#59dab2", "#31c194", "#25a27b", "#188463", "#11684d"]
custom_theme = pio.templates["plotly_dark"]
custom_theme.layout.update({"colorway": colors})
pio.templates.default = custom_theme

In [4]:
header = html.Div(
id="header",
    children=[
        html.H4("Website traffic analytics"),
        html.P(
        id="description",
            children="E-commerce website key variables evolution"
        )
    ]
)

In [5]:
visits_per_day=dataset.value_counts("date", sort=False)
visits_per_day.head()

date
2021-10-01    307
2021-10-02    308
2021-10-03    342
2021-10-04    325
2021-10-05    331
Name: count, dtype: int64

In [6]:
trend_title = html.P(className="chart-header", children="Website visits per day")

In [7]:
trend_figure=px.line(visits_per_day, markers=True)
trend_figure.layout.update(showlegend=False)
# trend_figure.show()
trend_graph = dcc.Graph(figure=trend_figure, className="graph")

In [8]:
first_row = html.Div(
className="graph-row",
    children=[
        html.Div(
        className="graph-container",
            children=[trend_title, trend_graph]
        )
    ]
)

In [9]:
funnel_title = html.P(className="chart-header", children="Sales funnel")

In [10]:
@app.callback(Output("funnel-graph", "figure"), [Input("os-dropdown", "value"), Input("country-dropdown", "value")])
def update_funnel_graph(os, country):
    subset = dataset
    if os:
        subset=subset.query(f"os=='{os}'")
    if country:
        subset=subset.query(f"contry=='{country}'")
    visits_per_page = subset.value_counts("page")
    fig=px.funnel(visits_per_page, color=visits_per_page)
    return fig

In [11]:
os_dropdown = dcc.Dropdown(
id="os-dropdown",
    placeholder="Filter by operating system",
    options=[
        {"label": "Windows", "value": "Windows"},
         {"label": "Android", "value": "Android"},
         {"label": "Mac", "value": "Mac"},
         {"label": "iOS", "value": "iOS"},
         {"label": "Linux", "value": "Linux"}
    ]
)

contry_dropdown = dcc.Dropdown(
id="country-dropdown",
    placeholder="Filter by country",
    options=[
        {"label": "United States", "value": "United States"},
        {"label": "Canada", "value": "Canada"}
            ]
)

In [12]:
funnel_controls=html.Div(
    className="controler-row",
    children=["os-dropdown", "country-dropdown"]
)

In [13]:
funnel_graph=dcc.Graph(id="funnel-graph", className="graph")

In [14]:
pie_title=html.P(className="chart-header", children="Visit distribution")

In [15]:
@app.callback(Output("pie-graph", "figure"), [Input("radio-buttons", "value")])
def update_pie_chart(value):
    fig=px.pie(dataset, names=value)
    return fig

In [16]:
radio_buttons = dcc.RadioItems(
id="radio-buttons",
    options=[
        {"label": "Operating System", "value": "os"},
         {"label": "Operating System", "value": "country"}
    ],
    value="os"
)

In [17]:
pie_controls = html.Div(
    className="controler-row",
    children=[radio_buttons]
)

In [18]:
pie_graph=dcc.Graph(id="pie-graph")

In [19]:
second_row=html.Div(
    className="graph-row",
    children=[
        html.Div(
            className="graph-container",
            children=[funnel_title, funnel_controls, funnel_graph]
        ),
                html.Div(
            className="graph-container",
            children=[pie_title, pie_controls, pie_graph]
        )
    ]
)

In [20]:
visits_day_time=dataset.groupby(["weekday", "time"]).size().reset_index(name="counts")
visits_day_time.head()

,weekday,time,counts
0,Friday,00:00:00,44
1,Friday,01:00:00,48
2,Friday,02:00:00,53
3,Friday,03:00:00,45
4,Friday,04:00:00,55


In [21]:
visits_day_time=visits_day_time.pivot_table(values="counts", index="time", columns="weekday")
visits_day_time.head()

weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
time,,,,,,,
00:00:00,44.0,38.0,42.0,39.0,47.0,41.0,43.0
01:00:00,48.0,46.0,53.0,48.0,46.0,53.0,45.0
02:00:00,53.0,44.0,58.0,46.0,51.0,44.0,37.0
03:00:00,45.0,35.0,50.0,54.0,35.0,39.0,66.0
04:00:00,55.0,46.0,43.0,51.0,41.0,45.0,46.0


In [22]:
heatmap_title=html.P(className="chart-header", children="Key activity periods")

In [23]:
heatmap_figure=go.Figure(
data=go.Heatmap(
    x=visits_day_time.columns.tolist(),
    y=visits_day_time.index.tolist(),
    z=visits_day_time.values.tolist(),
    autocolorscale=True,
    xgap=4, ygap=1
)
)
layout=heatmap_figure.layout.yaxis.update(autorange="reversed", tickvals=[0,6,12,18], 
                                         ticktext=["12 AM", "6 AM", "12 PM", "6 PM"])

In [24]:
heatmap_graph=dcc.Graph(figure=heatmap_figure)

In [25]:
country_title = html.P(className="chart-header", children="Visits by country")

In [26]:
country_figure = px.histogram(dataset, x="country", color="country")
layout = country_figure.layout.update(showlegend=False)

In [27]:
country_graph = dcc.Graph(figure=country_figure, className="graph")

In [28]:
third_row=html.Div(
    children=[
                html.Div(
            className="graph-container",
            children=[heatmap_title, heatmap_graph]
        ),
                        html.Div(
            className="graph-container",
            children=[country_title, country_graph]
        ),
    ]
)

In [29]:
app.layout = html.Div(
    id="root",
children=[header, 
          first_row, second_row, third_row
         ]
)

if __name__ == '__main__':
    app.run(mode='inline',  debug=True)